In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import statsmodels.api as sm
import statsmodels.formula.api as smf
from scipy import stats
import os

In [15]:
os.chdir(r'G:\My Drive\Master\Documents\Academia\Columbia-Postgraduate\QMSS\Res_unit_1\Time series\Assignments\Lab 2') # change working directory
pan = pd.read_csv("panel-for-R.csv")

In [16]:
variables = ['marital', 'age', 'year', 'sex', 'idnum', 'panelwave', 'degree', 'region']
sub = pan[variables].copy()

sub['av'] = np.where(pd.isna(sub['marital']), np.nan, 
                     np.where(sub['marital']==1, 1, 0))

In [17]:
# drop people working full-time in wave 1
sub['drop'] = np.where((sub['av']==1) & (sub['panelwave']==1), 1, 0)

In [18]:
# get idnum of subjects we're going to drop
drop_idnum = sub.idnum[sub['drop']==1].unique()

In [19]:
# subset the sub dataframe to include subjects whose idnum is not in the drop_idnum list
subpt = sub.loc[~sub['idnum'].isin(drop_idnum)].copy()

In [21]:
subpt['ytwo'] = np.where(pd.isna(subpt['av']), np.nan, 
                         np.where((subpt['av']==1) & (subpt['panelwave']==2), 2, 0))
subpt_two = subpt.loc[subpt['ytwo']==2, 'idnum'].unique()
subpt['ytwo_two'] = np.where(subpt['idnum'].isin(subpt_two), 2, 0)

subpt['ythree'] = np.where(pd.isna(subpt['av']), np.nan, 
                         np.where((subpt['av']==1) & (subpt['panelwave']==3), 3, 0))
subpt_three = subpt.loc[subpt['ythree']==3, 'idnum'].unique() 
subpt['ythree_three'] = np.where(subpt['idnum'].isin(subpt_three), 3, 0)

In [22]:
subpt.loc[:, 'combo'] = subpt['ytwo_two'].astype(str) + subpt['ythree_three'].astype(str)

In [23]:
subpt['combo'].value_counts()

00    2847
23     138
03     102
20      87
Name: combo, dtype: int64

In [24]:
subpt.groupby(['av', 'combo']).size()

av   combo
0.0  00       2151
     03         68
     20         42
     23         46
1.0  03         34
     20         29
     23         92
dtype: int64

In [25]:
# (slide 43)
subpt.loc[(subpt['combo']=='20') & (subpt['year']==2010), 'av'] = np.nan
subpt.loc[(subpt['combo']=='23') & (subpt['year']==2010), 'av'] = np.nan
subpt.groupby(['av', 'combo']).size()

av   combo
0.0  00       2151
     03         68
     20         29
     23         46
1.0  03         34
     20         29
     23         46
dtype: int64

In [46]:
lo2 = smf.glm('av ~ C(nyear)', 
              data = subpt[(subpt.year > 2006) & (subpt.age > 18)],
              family = sm.families.Binomial()).fit()
lo2.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                 Generalized Linear Model Regression Results                  
==============================================================================
Dep. Variable:                     av   No. Observations:                 1330
Model:                            GLM   Df Residuals:                     1328
Model Family:                Binomial   Df Model:                            1
Link Function:                  Logit   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -373.81
Date:                Sat, 12 Nov 2022   Deviance:                       747.61
Time:                        17:18:01   Pearson chi2:                 1.33e+03
No. Iterations:                     6   Pseudo R-squ. (CS):           0.004911
Covariance Type:            nonrobust                                         
===================================================================================
                      coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------
Intercept          -2.2163      0.122    -18.226      0.000      -2.455      -1.978
C(nyear)[T.4.0]    -0.5359      0.215     -2.496      0.013      -0.957      -0.115
===================================================================================
"""

In [47]:
# slide 47
np.exp(lo2.params)

Intercept          0.109012
C(nyear)[T.4.0]    0.585166
dtype: float64

In [43]:
lo3 = smf.glm('av ~ C(nyear) + sex + age + degree', 
              data = subpt[(subpt.year > 2006) & (subpt.age > 18)],
              family = sm.families.Binomial()).fit()
lo3.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                 Generalized Linear Model Regression Results                  
==============================================================================
Dep. Variable:                     av   No. Observations:                 1330
Model:                            GLM   Df Residuals:                     1325
Model Family:                Binomial   Df Model:                            4
Link Function:                  Logit   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -342.61
Date:                Sat, 12 Nov 2022   Deviance:                       685.22
Time:                        17:15:35   Pearson chi2:                 1.26e+03
No. Iterations:                     6   Pseudo R-squ. (CS):            0.05051
Covariance Type:            nonrobust                                         
===================================================================================
                      coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------
Intercept           0.0266      0.439      0.060      0.952      -0.834       0.887
C(nyear)[T.4.0]    -0.4490      0.220     -2.043      0.041      -0.880      -0.018
sex                -0.3062      0.206     -1.487      0.137      -0.710       0.097
age                -0.0478      0.007     -6.612      0.000      -0.062      -0.034
degree              0.1439      0.086      1.671      0.095      -0.025       0.313
===================================================================================
"""

In [42]:
lo4 = smf.glm('av ~ C(nyear)*C(sex) + age + degree', 
              data = subpt[(subpt.year > 2006) & (subpt.age > 18)],
              family = sm.families.Binomial()).fit()
lo4.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                 Generalized Linear Model Regression Results                  
==============================================================================
Dep. Variable:                     av   No. Observations:                 1330
Model:                            GLM   Df Residuals:                     1324
Model Family:                Binomial   Df Model:                            5
Link Function:                  Logit   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -342.51
Date:                Sat, 12 Nov 2022   Deviance:                       685.02
Time:                        17:14:59   Pearson chi2:                 1.26e+03
No. Iterations:                     6   Pseudo R-squ. (CS):            0.05065
Covariance Type:            nonrobust                                         
=================================================================================================
                                    coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------------------------
Intercept                        -0.3141      0.346     -0.907      0.364      -0.993       0.365
C(nyear)[T.4.0]                  -0.3509      0.309     -1.137      0.256      -0.956       0.254
C(sex)[T.2.0]                    -0.2425      0.250     -0.968      0.333      -0.733       0.248
C(nyear)[T.4.0]:C(sex)[T.2.0]    -0.1961      0.439     -0.446      0.655      -1.057       0.665
age                              -0.0478      0.007     -6.608      0.000      -0.062      -0.034
degree                            0.1446      0.086      1.678      0.093      -0.024       0.314
=================================================================================================
"""

In [48]:
subpt['av'] = np.where(pd.isna(subpt['marital']), np.nan, 
                     np.where(subpt['marital']==1, 1, 0))

In [49]:
subpt['avt'] = np.where((((subpt['combo']=='20') & (subpt['year']==2010)) | 
                        ((subpt['combo']=='23') & (subpt['year']==2010))), 1,
                       np.where((((subpt['combo']=='20') & (pd.isna(subpt['year'])))| 
                        ((subpt['combo']=='23') & (pd.isna(subpt['year'])))), np.nan, 0))

In [51]:
lo5 = smf.glm('av ~ C(nyear) + avt', data = subpt[subpt['year']>2006], family = sm.families.Binomial()).fit()
lo5.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                 Generalized Linear Model Regression Results                  
==============================================================================
Dep. Variable:                     av   No. Observations:                 1405
Model:                            GLM   Df Residuals:                     1402
Model Family:                Binomial   Df Model:                            2
Link Function:                  Logit   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -406.31
Date:                Sat, 12 Nov 2022   Deviance:                       812.63
Time:                        17:34:40   Pearson chi2:                 1.40e+03
No. Iterations:                     6   Pseudo R-squ. (CS):             0.1095
Covariance Type:            nonrobust                                         
===================================================================================
                      coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------
Intercept          -2.2307      0.122    -18.358      0.000      -2.469      -1.993
C(nyear)[T.4.0]    -0.5326      0.215     -2.482      0.013      -0.953      -0.112
avt                 4.0270      0.360     11.172      0.000       3.321       4.734
===================================================================================
"""